In [ ]:
from itertools import accumulate
import numpy as np
from tqdm.auto import tqdm

from icecream import ic

from matplotlib import pyplot as plt

In [ ]:
# Possible combinations of sets
# size: 100, sets: 10, density: .2
# size: 1000, sets: 100, density: .2
# size: 10000, sets: 1000, density: .2
# size: 100000, sets: 10000, density: .1
# size: 100000, sets: 10000, density: .2
# size: 100000, sets: 10000, density: .3

UNIVERSE_SIZE = 100
NUM_SETS = 10
DENSITY = 0.2

rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))

In [ ]:
# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.pow(SETS.sum(axis=1), 1.1)

## Helper Functions

In [ ]:
def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    phenotype = np.logical_or.reduce(SETS[solution])
    return np.all(phenotype)


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()

In [ ]:
def fitness(solution: np.ndarray):
    return valid(solution), -cost(solution)

# Steepest step and restart

Modified section:

I completely ignored the last part of the first review (advising to make more
variables dynamic) because, by doing so, the solutions were just worse...
The main modification was introducing a dynamic mutation rate controlled
by a learning rate and the no improvement treshold constants.
While I was reviewing again, after the second review came in, I realised I had
to decide whether to adapt the learning rate (as I already did) or to restart
after a given number of steps saw no improvement.
Not only I had already implement the first option, but I also found it more
reasonable since I wanted to explore also less promising areas.
In the end, I just added some comments and refactored the code as the second
review pointed out how messy my code was...

In [ ]:
NUM_RESTARTS = 3  # Restarting to avoid local optima
STEEPEST_STEP_CANDIDATES = 5  # N° of solutions evaluated at each step
MAX_STEPS = 20000  # If NUM_RESTARTS controls the number of cycles of the outer loop, this one controls the maximum number of cycles of the inner one
TRUE_MAX_STEPS = MAX_STEPS // STEEPEST_STEP_CANDIDATES  # Adjusted max steps considering also the n° of candidates evaluated at each step
LEARNING_RATE = 0.01  # Mutation rate adjustment rate
NO_IMPROVEMENT_THRESHOLD = 100  # N° of cycles with no improvement before increasing the mutation rate

# Function to dynamically adjust mutation rate
def adjust_mutation_rate(current_rate, no_improvement_count):
    """If the solution stops improving for NO_IMPROVEMENT_THRESHOLD
    cycles, the mutation rate is boosted, therefore the
    exploration aspect of the algorithm.
    Otherwise, if the solution is continuously improving, the
    mutation rate is lowered to boost the exploitation part."""

    if no_improvement_count > NO_IMPROVEMENT_THRESHOLD:
        return min(1.0, current_rate + LEARNING_RATE)
    return max(0.01, current_rate - LEARNING_RATE)


# Modified multiple_mutation function with dynamic mutation rate
def multiple_mutation(solution: np.ndarray, mutation_rate: float) -> np.ndarray:
    """This function flips a number of sets of a given solution based on the mutation rate."""
    new_sol = solution.copy()
    mask = rng.random(NUM_SETS) < mutation_rate
    return np.logical_xor(new_sol, mask)


max_value = (0, -1)
num_steps = 0
history = list()  # Used only for visualization
best_solution = []

# The algorithm is restarted more times to avoid local optima (essence of the algorithm itself)
for i in tqdm(range(NUM_RESTARTS), position=0):
    solution = rng.random(NUM_SETS) < .5
    history.append(fitness(solution)[1])

    # Variables needed for the new dynamic mutation rate
    last_improvement = (0, 0)
    new_solution = solution
    no_improvement_count = 0

    # In this version the mutation rate becomes dynamic, being adusted based on the performance of the algorithm
    # It defines the number of sets being inverted during the mutation phase for each of the selected candidates
    mutation_rate = 0.01

    for n in tqdm(range(TRUE_MAX_STEPS // NUM_RESTARTS), position=1, desc=f'step {i+1}'):
        # create candidate solutions and evaluate them
        candidates = [multiple_mutation(solution, mutation_rate) for _ in range(STEEPEST_STEP_CANDIDATES)]
        candidates_fitness = list()
        for c in candidates:
            f = fitness(c)
            history.append(f[1])
            candidates_fitness.append(f)
        idx = candidates_fitness.index(max(candidates_fitness))

        # keep candidate solution that yielded the steepest ascent
        new_solution = candidates[idx]
        new_fitness = candidates_fitness[idx]
        num_steps += STEEPEST_STEP_CANDIDATES

        if new_fitness > fitness(solution):
            solution = new_solution
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        # Adjust mutation rate based on improvement
        mutation_rate = adjust_mutation_rate(mutation_rate, no_improvement_count)

    ic(fitness(solution))

    if fitness(solution) > max_value:
        max_value = fitness(solution)
        best_solution = solution


ic(fitness(best_solution))

plt.figure(figsize=(14, 8))
plt.plot(
    range(len(history)),
    list(accumulate(history, max)),
    color="red",
)
plt.scatter(range(len(history)), history, marker=".")